In [ ]:
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings('ignore')
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
plt.style.use('ggplot')

In [ ]:
Traindata_with_Target = pd.read_csv("Traindata_with_Target.csv")
Train_Demographics = pd.read_csv("Train_Demographics.csv")
Train_Vehicle = pd.read_csv("Train_Vehicle.csv")
Train_Policy = pd.read_csv("Train_Policy.csv")
Train_Claim = pd.read_csv("Train_Claim.csv")

In [ ]:
a = Traindata_with_Target.merge(Train_Demographics, how='left', on='CustomerID')

In [ ]:
b = a.merge(Train_Policy,how='left',on='CustomerID')

In [ ]:
c = b.merge(Train_Claim,how='left',on='CustomerID')

In [ ]:
train_data = c.merge(Train_Vehicle,how='left',on = 'CustomerID')

In [ ]:
Test = pd.read_csv("Test.csv")
Test_Demographics = pd.read_csv("Test_Demographics.csv")
Test_Vehicle = pd.read_csv("Test_Vehicle.csv")
Test_Policy = pd.read_csv("Test_Policy.csv")
Test_Claim = pd.read_csv("Test_Claim.csv")

In [ ]:
e = Test.merge(Test_Demographics, how='left', on='CustomerID')
f = e.merge(Test_Policy, how='left', on='CustomerID')
g = f.merge(Test_Claim, how='left', on='CustomerID')
test_data = g.merge(Test_Vehicle, how='left', on='CustomerID')

In [ ]:
train_data.shape

In [ ]:
test_data.shape

In [ ]:
pivoted_train = train_data.pivot(index='CustomerID',columns='VehicleAttribute',values='VehicleAttributeDetails').reset_index()
pivoted_test = test_data.pivot(index='CustomerID',columns='VehicleAttribute',values='VehicleAttributeDetails').reset_index()

In [ ]:
Train = train_data.merge(pivoted_train,how='left',on='CustomerID')
Test = test_data.merge(pivoted_test,how='left',on='CustomerID')

In [ ]:
# Dropping Country Column from Train and Test Data Because it has only one value - India
Train.drop(["Country",'VehicleAttribute','VehicleAttributeDetails'],axis=1,inplace=True)
Test.drop(["Country",'VehicleAttribute','VehicleAttributeDetails'],axis=1,inplace=True)

In [ ]:
Train = Train.drop_duplicates()
Test = Test.drop_duplicates()

In [ ]:
Train.to_excel("Train_Data.xlsx",sheet_name='Train_Data')
Test.to_excel("Test_data.xlsx",sheet_name='Test_Data')

In [ ]:
Train = pd.read_excel('Train_Data.xlsx',
                      na_values=['NA','-1','?','???','MISSINGVAL','MISSINGVALUE','MISSEDDATA','-5'])

In [ ]:
Test = pd.read_excel('Test_data.xlsx',
                     na_values=['NA','-1','?','???','MISSINGVAL','MISSINGVALUE','MISSEDDATA','-5'])

In [ ]:
import missingno as msno
msno.bar(Train)
plt.show()

In [ ]:
msno.bar(Test)
plt.show()

In [ ]:
# Treating the NA Values
# Type of collision column has 20648 Na's but when comparing the target value with this column 
# Not Fraud has a majority value 
# My thought is to filter based on the Target values and then impute with mode for this column

In [ ]:
train_fraud_yes = Train[Train['ReportedFraud'] == 'Y']
train_fraud_no = Train[Train['ReportedFraud'] == 'N']

In [ ]:
train_fraud_yes['TypeOfCollission'] = train_fraud_yes['TypeOfCollission'].fillna(train_fraud_yes['TypeOfCollission'].mode()[0])
train_fraud_no['TypeOfCollission'] = train_fraud_no['TypeOfCollission'].fillna(train_fraud_no['TypeOfCollission'].mode()[0])

In [ ]:
Train = train_fraud_yes.append(train_fraud_no, ignore_index=True)

In [ ]:
Train['PropertyDamage'] = Train['PropertyDamage'].fillna("NO")
Train['PoliceReport'] = Train['PoliceReport'].fillna("NO")
# Train['InsuredGender'] = Train['InsuredGender'].fillna(Train['InsuredGender'].mode()[0])
# Train['PolicyAnnualPremium'] = Train['PolicyAnnualPremium'].fillna(Train['PolicyAnnualPremium'].median())

In [ ]:
Train.dropna(inplace=True)

In [ ]:
Train.shape

In [ ]:
Test['TypeOfCollission'] = Test['TypeOfCollission'].fillna(Test['TypeOfCollission'].mode()[0])
Test['TypeOfCollission'] = Test['TypeOfCollission'].fillna(Test['TypeOfCollission'].mode()[0])
Test['PropertyDamage'] = Test['PropertyDamage'].fillna("NO")
Test['PoliceReport'] = Test['PoliceReport'].fillna("NO")

In [ ]:
Test.dropna(inplace=True)

In [ ]:
Test.shape

In [ ]:
Train.drop(["Unnamed: 0"],axis=1,inplace=True)
Test.drop(["Unnamed: 0"],axis=1,inplace=True)

In [ ]:
Train.columns

In [ ]:
#plt.figure(figsize=[5,4])
fig = px.pie(Train,names='ReportedFraud')
fig.show()

In [ ]:
# checking for multicollinearity
plt.figure(figsize = (18, 12))

corr = Train.corr()
mask = np.triu(np.ones_like(corr, dtype = bool))

sns.heatmap(data = corr, mask = mask, annot = True, fmt = '.2g', linewidth = 1)
plt.show()

In [ ]:
fig = px.box(Train, x="InsurancePolicyState", y="AmountOfTotalClaim", color="ReportedFraud")
fig.show()

In [ ]:
Train.columns

In [ ]:
Train['count'] = 1
groupdf_is = Train.groupby(by=['InsuredGender','ReportedFraud']).count()
groupdf_is.reset_index(inplace=True)
groupdf_is = groupdf_is[['InsuredGender','ReportedFraud','count']]
groupdf_is.head()

In [ ]:
fig=px.bar(groupdf_is,x='InsuredGender',y='count',color='ReportedFraud',text='count',
           pattern_shape_sequence=['|','/'],pattern_shape='ReportedFraud')
fig.show()

In [ ]:
Train.columns

In [ ]:
groupdf_el = Train.groupby(by=['InsuredEducationLevel','ReportedFraud']).count()
groupdf_el.reset_index(inplace=True)
groupdf_el = groupdf_el[['InsuredEducationLevel','ReportedFraud','count']]
groupdf_el.head()

In [ ]:
fig = px.bar(groupdf_el,x='InsuredEducationLevel',y='count',color='ReportedFraud',text='count',title="Insurance Claim for different education Level")
fig.show()

In [ ]:
groupdf_ic = Train.groupby(by=['InsuredOccupation','ReportedFraud']).count()
groupdf_ic.reset_index(inplace=True)
groupdf_ic = groupdf_ic[['InsuredOccupation','ReportedFraud','count']]
groupdf_ic.head()

In [ ]:
fig = px.bar(groupdf_ic,x='InsuredOccupation',y='count',color='ReportedFraud',text='count',title='Number of Claims different Occupation')
fig.show()

In [ ]:
groupdf_h = Train.groupby(by=['InsuredHobbies','ReportedFraud']).count()
groupdf_h.reset_index(inplace=True)
groupdf_h = groupdf_h[['InsuredHobbies','ReportedFraud','count']]
groupdf_h.head()

In [ ]:
fig = px.bar(groupdf_h,x='InsuredHobbies',y='count',color='ReportedFraud',text='count',title="Hobby Wise claim")
fig.show()

In [ ]:
groupdf_rl = Train.groupby(by=['InsuredRelationship','ReportedFraud']).count()
groupdf_rl.reset_index(inplace=True)
groupdf_rl = groupdf_rl[['InsuredRelationship','ReportedFraud','count']]
groupdf_rl.head()

In [ ]:
fig = px.bar(groupdf_rl,x='InsuredRelationship',y='count',color='ReportedFraud',text='count')
fig.show()

In [ ]:
Train.columns

In [ ]:
groupdf_it = Train.groupby(by=['TypeOfIncident','ReportedFraud']).count()
groupdf_it.reset_index(inplace=True)
groupdf_it = groupdf_it[['TypeOfIncident','ReportedFraud','count']]
groupdf_it.head()

In [ ]:
fig = px.bar(groupdf_it,x='TypeOfIncident',y='count',color='ReportedFraud',text='count',
             title="Number of claims Vs. Incident type")
fig.show()

In [ ]:
groupdf_ct = Train.groupby(by=['TypeOfCollission','ReportedFraud']).count()
groupdf_ct.reset_index(inplace=True)
groupdf_ct = groupdf_ct[['TypeOfCollission','ReportedFraud','count']]
groupdf_ct.head()

In [ ]:
fig = px.bar(groupdf_ct,x='TypeOfCollission',y='count',color='ReportedFraud',text='count',title="Nos. of Claim for different type of collision")
fig.show()

In [ ]:
groupdf_ins = Train.groupby(by=['SeverityOfIncident','ReportedFraud']).count()
groupdf_ins.reset_index(inplace=True)
groupdf_ins = groupdf_ins[['SeverityOfIncident','ReportedFraud','count']]
groupdf_ins.head()

In [ ]:
fig = px.bar(groupdf_ins,x='SeverityOfIncident',y='count',color='ReportedFraud',text='count',title="Number of claim Vs. Incident severity")
fig.show()

In [ ]:
Train.columns

In [ ]:
groupdf_ac = Train.groupby(by=['AuthoritiesContacted','ReportedFraud']).count()
groupdf_ac.reset_index(inplace=True)
groupdf_ac = groupdf_ac[['AuthoritiesContacted','ReportedFraud','count']]
groupdf_ac.head()

In [ ]:
fig = px.bar(groupdf_ac,x='AuthoritiesContacted',y='count',color='ReportedFraud',text='count')
fig.show()

In [ ]:
groupdf_st = Train.groupby(by=['IncidentState','ReportedFraud']).count()
groupdf_st.reset_index(inplace=True)
groupdf_st = groupdf_st[['IncidentState','ReportedFraud','count']]
groupdf_st.head()

In [ ]:
fig = px.bar(groupdf_st,x='IncidentState',y='count',color='ReportedFraud',text='count',title="Claims for different state")
fig.show()